# DASCore Training

December 2, 2024

This notebook shows an example DASCore application: tracking traffic in Salt Lake City. These signals are likely dominated by the light-rail trains. 

The data were collected in May of 2023. The temporal sampling rate was ~650Hz before down-sampling to 4 Hz for this tutorial. 

<a target="_blank" href="https://colab.research.google.com/github/DASDAE/seg_tutorial/blob/master/04_training.ipynb">

</a>  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

#### Useful links: 
* [Colab link](https://colab.research.google.com/github/DASDAE/seg_tutorial/blob/master/04_training.ipynb)
* [DASCore tutorial](https://dascore.org/tutorial/concepts.html)
* [Salt Lake City public transportation map](https://www.rideuta.com/-/media/Files/Rider-Tools/System-Maps/SLCountySystemMapAugust2024.pdf)
  

In [ ]:
%%capture

# First ensure DASCore is installed. If not, install and restart the kernel.
try:
    import dascore as dc
except ImportError:
    !pip install dascore
    !pip install ipympl
    # resetart kernel
    import IPython
    IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

In [ ]:
import numpy as np

## Setup
First, we create a directory of DAS files to simulate the output of an acquisition. 

In [ ]:
# Create a directory of DAS file
from pathlib import Path

path = Path("UoU_data")
path.mkdir(exist_ok=True)

# Get a spool of data with 5 minute patches
spool = dc.get_example_spool("UoU_lf_urban.hdf5").chunk(time=60*5)
for num, patch in enumerate(spool):
    patch_path = (path / f"UoU_{num}.hdf5")
    if patch_path.exists():
        continue
    patch.io.write(patch_path, "dasdae")

## Exploration

We initialize a spool on the directory of DAS files and explore a summary of the contents.

In [ ]:
spool = dc.spool(path)

In [ ]:
spool.get_contents()

There are 10 files, each of 5 minutes of data. One example file looks like this:

In [ ]:
patch = spool[2]
patch.viz.waterfall(scale=0.05);

Notice the linear features pre-dominantly moving from left to right. 

### **Exercise** (Train Speed)


Estimate the train speed. You could just zoom in around the train signals, use the interactive mode of Matplotlib, or perform and plot a [2D dft](https://dascore.org/api/dascore/transform/fourier/dft.html).

## Aggregations

Let's calculate some statistics of the data. For example, the standard deviation of each channel (distance dimension) for each of the files.

In [ ]:
abs_max = [
    x.abs()
    .aggregate(dim="time", method=np.max, dim_reduce="first") 
    for x in spool
]

In [ ]:
patch = (
    # Merge all the aggregate patches together
    dc.spool(abs_max).chunk(time=None)[0]
    # Ensure data type reflects the new meaning
    .update_attrs(data_type="maximum absolute values")
)

In [ ]:
patch.viz.waterfall()

### **Exercise** (Aggregations)
Redo the aggregations above, but calculate the standard deviation for every 60 seconds of data. 


## Moving windows

A similar concept to the aggregations is that of a moving window which is implemented with DASCore's [`Patch.rolling`](https://dascore.org/api/dascore/proc/rolling/rolling.html). These are more efficient for overlapping/dense windows. 


In [ ]:
spool_mw = dc.spool(path).chunk(time=120, overlap=20, tolerance=10)

output = []
for patch in spool_mw:
    std_patch = (
        patch.abs()
        .rolling(center=True, time=10, step=1)
        .std()
        .dropna("time")
    )
    
    output.append(std_patch.dropna("time"))


In [ ]:
patch = (
    dc.spool(output)
    .chunk(time=None)[0]
    .update_attrs(data_type="standard deviation")
)

In [ ]:
patch.viz.waterfall(scale=0.5);

### **Exercise** (Rolling)
Use `Patch.rolling` to create a simple STA/LTA detector.


## Final Challenge
Determine the power in the 0.1, 0.5, and 1Hz frequency bins (with +/- .1Hz) for each 60 seconds of the data. Plot the result with time on the x axis, distance on the Y, and the data as the power. 